# GAN

*generative adversarial networks* (敵対的生成ネットワーク)

<br>

GANをpytorchで実装する

とりあえずMNIST


---

## 必要なもの

### ライブラリ

In [26]:
import torch
from torch import nn, optim
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (8, 4)

### MNISTデータセット

In [19]:
from torchvision.datasets import MNIST
from torchvision import transforms

train_data = MNIST(
    root='./data',
    train=True,
    transform=transforms.ToTensor(),
    download=True)

test_data = MNIST(
    root='./data',
    train=False,
    transform=transforms.ToTensor(),
    download=True)


X_train = train_data.data.to(torch.float)
y_train = train_data.targets

X_test = test_data.data.to(torch.float)
y_test = test_data.targets


---

## generator

生成機

ノイズ(10次元ベクトル)から784次元のベクトルを生成する。  
出力値は各ピクセルの濃淡を $0 \sim 1$ で表すので`sigmoid`関数をかける。

In [21]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(10, 128),
            nn.ReLU(),
            nn.Linear(128, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 28*28),
            nn.Sigmoid()
        )

    def forward(self, x):
        self.network(x)

generator = Generator()


---

## discriminator

識別器。偽物である確率を出力する。

In [23]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        self.network(x)

discriminator = Discriminator()


---

## 学習



In [13]:
batch_size = 128
train_loader = torch.utils.data.DataLoader(
    dataset=train_data,
    batch_size=batch_size,
    shuffle=True)

### discriminator

識別器の学習

In [29]:
def train_disc(model, data_loader, optimizer):
    model.train()
    for (X, label) in data_loader:
        y = model(X)
        loss = F.mse_loss(label, y)
        loss.backward()
        optimizer.step()
    return loss.item()

### generator

生成機の学習。誤差関数は出力画像を識別器に入れた値(偽物である確率)。

In [67]:
def train_gen(model, optim, discriminator, epochs, batch_size):
    model.train()
    for _ in range(epochs):
        noises = torch.randn(batch_size, 10) # バッチサイズ分のノイズを生成
        y = model(noises) # ノイズから画像を生成
        loss = discriminator(y).mean() # 偽物である確率を計算
        loss.backward()
        optim.step()
    return y

### 敵対

二つを学習させる